In [1]:
import tensorflow as tf
from data_loader import load_and_prepare_data
from model_builder import build_cnn_model
from train_evaluate import train_and_evaluate_model
from forward_propagation import *
import numpy as np
import os

### Load Data dan Train Model

In [3]:
EPOCHS = 10
BATCH_SIZE = 128
NUM_CLASSES = 10 
RESULTS_DIR = "../results"
if not os.path.exists(RESULTS_DIR):
    os.makedirs(RESULTS_DIR)
SCORES_FILE = os.path.join(RESULTS_DIR, "scores.txt")

In [4]:
def log_results(file_path, experiment_name, f1_score_val, loss_val, acc_val, details=""):
    with open(file_path, "a") as f:
        f.write(f"Eksperimen: {experiment_name}\n")
        if details:
            f.write(f"  Details: {details}\n")
        f.write(f"  Macro F1-Score: {f1_score_val:.4f}\n")
        f.write(f"  Test Loss: {loss_val:.4f}\n")
        f.write(f"  Test Accuracy: {acc_val:.4f}\n")
        f.write("-" * 30 + "\n")

In [ ]:
def run_experiments():
    (x_train, y_train), (x_val, y_val), (x_test, y_test) = load_and_prepare_data()
    input_shape = x_train.shape[1:]

    all_results = {} 


    if os.path.exists(SCORES_FILE):
        os.remove(SCORES_FILE)

    # --- 1. Pengaruh Jumlah Layer Konvolusi ---
    print("\n=== Eksperimen: Pengaruh Jumlah Layer Konvolusi ===")
    num_conv_layers_variations = [
        ("1_conv_layers", [{'filters': 32, 'kernel_size': (3,3), 'activation': 'relu'}]),
        ("2_conv_layers", [{'filters': 32, 'kernel_size': (3,3), 'activation': 'relu'},
                           {'filters': 64, 'kernel_size': (3,3), 'activation': 'relu'}]),
        ("3_conv_layers", [{'filters': 32, 'kernel_size': (3,3), 'activation': 'relu'},
                           {'filters': 64, 'kernel_size': (3,3), 'activation': 'relu'},
                           {'filters': 128, 'kernel_size': (3,3), 'activation': 'relu'}])
    ]

    for name, config in num_conv_layers_variations:
        print(f"\n--- Variasi: {name} ---")
        model_name = f"cnn_{name}"
        model = build_cnn_model(input_shape, NUM_CLASSES, config, pooling_type='max')
        model.summary()
        _, loss, acc, f1 = train_and_evaluate_model(
            model, x_train, y_train, x_val, y_val, x_test, y_test,
            EPOCHS, BATCH_SIZE, model_name, RESULTS_DIR
        )
        all_results[model_name] = f1
        log_results(SCORES_FILE, model_name, f1, loss, acc, f"Config: {config}")
        tf.keras.backend.clear_session() # Reset state Keras

    # --- 2. Pengaruh Banyak Filter per Layer Konvolusi ---
    print("\n=== Eksperimen: Pengaruh Banyak Filter per Layer Konvolusi ===")
    #2 layer konvolusi sebagai dasar
    num_filters_variations = [
        ("filters_16_32", [{'filters': 16, 'kernel_size': (3,3)}, {'filters': 32, 'kernel_size': (3,3)}]),
        ("filters_32_64", [{'filters': 32, 'kernel_size': (3,3)}, {'filters': 64, 'kernel_size': (3,3)}]), # Default
        ("filters_64_128", [{'filters': 64, 'kernel_size': (3,3)}, {'filters': 128, 'kernel_size': (3,3)}])
    ]
    base_conv_layers_for_filters = 2 

    for name, config in num_filters_variations:
        print(f"\n--- Variasi: {name} ---")
        model_name = f"cnn_{name}"

        model = build_cnn_model(input_shape, NUM_CLASSES, config, pooling_type='max')
        model.summary()
        _, loss, acc, f1 = train_and_evaluate_model(
            model, x_train, y_train, x_val, y_val, x_test, y_test,
            EPOCHS, BATCH_SIZE, model_name, RESULTS_DIR
        )
        all_results[model_name] = f1
        log_results(SCORES_FILE, model_name, f1, loss, acc, f"Config: {config}")
        tf.keras.backend.clear_session()

    # --- 3. Pengaruh Ukuran Filter per Layer Konvolusi ---
    print("\n=== Eksperimen: Pengaruh Ukuran Filter per Layer Konvolusi ===")
    # 2 layer konvolusi dengan filter [32, 64] sebagai dasar
    kernel_size_variations = [
        ("kernel_3x3_3x3", [{'filters': 32, 'kernel_size': (3,3)}, {'filters': 64, 'kernel_size': (3,3)}]), # Default
        ("kernel_5x5_5x5", [{'filters': 32, 'kernel_size': (5,5)}, {'filters': 64, 'kernel_size': (5,5)}]),
        ("kernel_mix_3x3_5x5", [{'filters': 32, 'kernel_size': (3,3)}, {'filters': 64, 'kernel_size': (5,5)}])
    ]

    for name, config in kernel_size_variations:
        print(f"\n--- Variasi: {name} ---")
        model_name = f"cnn_{name}"
        model = build_cnn_model(input_shape, NUM_CLASSES, config, pooling_type='max')
        model.summary()
        _, loss, acc, f1 = train_and_evaluate_model(
            model, x_train, y_train, x_val, y_val, x_test, y_test,
            EPOCHS, BATCH_SIZE, model_name, RESULTS_DIR
        )
        all_results[model_name] = f1
        log_results(SCORES_FILE, model_name, f1, loss, acc, f"Config: {config}")
        tf.keras.backend.clear_session()

    # --- 4. Pengaruh Jenis Pooling Layer ---
    print("\n=== Eksperimen: Pengaruh Jenis Pooling Layer ===")
    # 2 layer [32,64] kernel (3,3)
    base_config_for_pooling = [
        {'filters': 32, 'kernel_size': (3,3), 'activation': 'relu'},
        {'filters': 64, 'kernel_size': (3,3), 'activation': 'relu'}
    ]
    pooling_types = [
        ("max_pooling", "max"),
        ("avg_pooling", "avg")
    ]

    for name, pool_type in pooling_types:
        print(f"\n--- Variasi: {name} ---")
        model_name = f"cnn_{name}"
        model = build_cnn_model(input_shape, NUM_CLASSES, base_config_for_pooling, pooling_type=pool_type)
        model.summary()
        _, loss, acc, f1 = train_and_evaluate_model(
            model, x_train, y_train, x_val, y_val, x_test, y_test,
            EPOCHS, BATCH_SIZE, model_name, RESULTS_DIR
        )
        all_results[model_name] = f1
        log_results(SCORES_FILE, model_name, f1, loss, acc, f"Pooling type: {pool_type}")
        tf.keras.backend.clear_session()

    print("\n=== Ringkasan F1-Score Semua Eksperimen ===")
    for model_name, f1_val in all_results.items():
        print(f"{model_name}: {f1_val:.4f}")

In [6]:
run_experiments()

Jumlah data training: 40000
Jumlah data validasi: 10000
Jumlah data test: 10000

=== Eksperimen: Pengaruh Jumlah Layer Konvolusi ===

--- Variasi: 1_conv_layers ---


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 32, 32, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     1,048,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,050,890 (4.01 MB)

 Trainable params: 1,050,890 (4.01 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 32, 32, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d               │ (None, 16, 16, 32)     │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 16, 16, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_1             │ (None, 8, 8, 64)       │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       524,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 545,098 (2.08 MB)

 Trainable params: 545,098 (2.08 MB)

 Non-trainable params: 0 (0.00 B)


--- Training Model: cnn_avg_pooling ---
Epoch 1/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 13s 38ms/step - accuracy: 0.3445 - f1_score_macro: 0.1950 - loss: 1.8137 - val_accuracy: 0.5224 - val_f1_score_macro: 0.1951 - val_loss: 1.3595
Epoch 2/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 12s 37ms/step - accuracy: 0.5432 - f1_score_macro: 0.1956 - loss: 1.2959 - val_accuracy: 0.5669 - val_f1_score_macro: 0.1953 - val_loss: 1.2114
Epoch 3/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 12s 39ms/step - accuracy: 0.5935 - f1_score_macro: 0.1956 - loss: 1.1567 - val_accuracy: 0.5678 - val_f1_score_macro: 0.1950 - val_loss: 1.2230
Epoch 4/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 12s 38ms/step - accuracy: 0.6234 - f1_score_macro: 0.1956 - loss: 1.0723 - val_accuracy: 0.6197 - val_f1_score_macro: 0.1953 - val_loss: 1.0810
Epoch 5/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 12s 37ms/step - accuracy: 0.6557 - f1_score_macro: 0.1956 - loss: 0.9799 - val_accuracy: 0.6440 - val_f1_score_macro: 0.1954 - val_loss: 1.0158
Epoch 6/10
313/313 ━━━━━━━━━━━━━━━━━━━━

### Forward Propagation

In [2]:
(_, _), (_, _), (x_test, y_test) = load_and_prepare_data()

num_test_samples = 10
x_test_sample = x_test[:num_test_samples]
y_test_sample_true_labels = y_test[:num_test_samples]

model_configs = [
    "cnn_1_conv_layers",
    "cnn_2_conv_layers",
    "cnn_3_conv_layers",
    "cnn_filters_16_32",
    "cnn_filters_32_64",
    "cnn_filters_64_128",
    "cnn_kernel_3x3_3x3",
    "cnn_kernel_mix_3x3_5x5",
    "cnn_kernel_5x5_5x5",
    "cnn_max_pooling",
    "cnn_avg_pooling"
]

for model_config in model_configs:
    print(f"\n\n======================================================================")
    print(f"--- Menguji Forward Propagation untuk Model: {model_config} ---")
    print(f"======================================================================")
    
    KERAS_WEIGHTS_PATH = f"../saved_models/{model_config}.weights.h5"

    if not os.path.exists(KERAS_WEIGHTS_PATH):
        print(f"ERROR: File bobot {KERAS_WEIGHTS_PATH} tidak ditemukan. ")
        continue 
    
    test_forward_propagation(x_test_sample, y_test_sample_true_labels,
                                KERAS_WEIGHTS_PATH,
                                model_config,
                                num_classes=10)
    print(f"--- Pengujian untuk Model: {model_config} Selesai ---")


Jumlah data training: 40000
Jumlah data validasi: 10000
Jumlah data test: 10000


--- Menguji Forward Propagation untuk Model: cnn_1_conv_layers ---


c:\Python312\Lib\site-packages\keras\src\saving\saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 303ms/step
Bobot dari ../saved_models/cnn_1_conv_layers.weights.h5 berhasil dimuat ke model Keras.
Custom CNN model built with layers from Keras model.

--- Starting Forward Propagation (from scratch) ---
--- Forward Propagation (from scratch) Finished ---

--- Perbandingan Hasil Prediksi ---
Sample true labels: [3 8 8 0 6 6 1 6 3 1]
Keras predicted labels: [3 8 0 8 4 6 1 6 3 1]
Scratch predicted labels: [3 8 0 8 4 6 1 6 3 1]

Probabilitas output Keras (sampel pertama, 5 output pertama): [1.2970270e-03 5.2811310e-04 2.5276563e-03 8.7561977e-01 9.4591286e-03]
Probabilitas output Scratch (sampel pertama, 5 output pertama): [1.29702729e-03 5.28113342e-04 2.52765645e-03 8.75619701e-01
 9.45912517e-03]

Macro F1-Score (Keras): 0.5500
Macro F1-Score (Scratch): 0.5500
Implementasi forward propagation from scratch KONSISTEN dengan Keras (berdasarkan F1-score).
--- Pengujian untuk Model: cnn_1_conv_layers Selesai ---


--- Menguji Forward Propagation untuk Model: cnn

c:\Python312\Lib\site-packages\keras\src\saving\saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Bobot dari ../saved_models/cnn_2_conv_layers.weights.h5 berhasil dimuat ke model Keras.
Custom CNN model built with layers from Keras model.

--- Starting Forward Propagation (from scratch) ---
--- Forward Propagation (from scratch) Finished ---

--- Perbandingan Hasil Prediksi ---
Sample true labels: [3 8 8 0 6 6 1 6 3 1]
Keras predicted labels: [3 8 0 0 6 6 1 6 3 9]
Scratch predicted labels: [3 8 0 0 6 6 1 6 3 9]

Probabilitas output Keras (sampel pertama, 5 output pertama): [5.6459107e-03 7.0143410e-04 4.8021260e-03 8.1240046e-01 3.8303102e-03]
Probabilitas output Scratch (sampel pertama, 5 output pertama): [5.64591351e-03 7.01434532e-04 4.80212599e-03 8.12400506e-01
 3.83031108e-03]

Macro F1-Score (Keras): 0.6667
Macro F1-Score (Scratch): 0.6667
Implementasi forward propagation from scratch KONSISTEN dengan Keras (berdasarkan F1-score).
--- Pengujian untuk Model: cnn_2_conv_layers Selesai ---


--- Menguji Forward Propagation untuk Model: cnn_3_conv_layers ---
1/1 ━━━━━━━━━━━━━━━━

...

Bobot dari ../saved_models/cnn_kernel_5x5_5x5.weights.h5 berhasil dimuat ke model Keras.
Custom CNN model built with layers from Keras model.

--- Starting Forward Propagation (from scratch) ---
--- Forward Propagation (from scratch) Finished ---

--- Perbandingan Hasil Prediksi ---
Sample true labels: [3 8 8 0 6 6 1 6 3 1]
Keras predicted labels: [3 8 8 0 4 6 1 2 3 1]
Scratch predicted labels: [3 8 8 0 4 6 1 2 3 1]

Probabilitas output Keras (sampel pertama, 5 output pertama): [2.1209412e-03 2.5074517e-03 2.4270684e-04 8.4597373e-01 3.1566314e-04]
Probabilitas output Scratch (sampel pertama, 5 output pertama): [2.12094146e-03 2.50745185e-03 2.42706862e-04 8.45973659e-01
 3.15662915e-04]

Macro F1-Score (Keras): 0.6429
Macro F1-Score (Scratch): 0.6429
Implementasi forward propagation from scratch KONSISTEN dengan Keras (berdasarkan F1-score).
--- Pengujian untuk Model: cnn_kernel_5x5_5x5 Selesai ---


--- Menguji Forward Propagation untuk Model: cnn_max_pooling ---
1/1 ━━━━━━━━━━━━━━━━

c:\Python312\Lib\site-packages\keras\src\saving\saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Bobot dari ../saved_models/cnn_max_pooling.weights.h5 berhasil dimuat ke model Keras.
Custom CNN model built with layers from Keras model.

--- Starting Forward Propagation (from scratch) ---
--- Forward Propagation (from scratch) Finished ---

--- Perbandingan Hasil Prediksi ---
Sample true labels: [3 8 8 0 6 6 1 6 3 1]
Keras predicted labels: [3 8 0 0 6 6 1 6 3 1]
Scratch predicted labels: [3 8 0 0 6 6 1 6 3 1]

Probabilitas output Keras (sampel pertama, 5 output pertama): [1.4297190e-04 3.9624637e-03 5.6304544e-04 8.8508511e-01 8.0866914e-04]
Probabilitas output Scratch (sampel pertama, 5 output pertama): [1.42971887e-04 3.96246111e-03 5.63045506e-04 8.85085069e-01
 8.08669797e-04]

Macro F1-Score (Keras): 0.8667
Macro F1-Score (Scratch): 0.8667
Implementasi forward propagation from scratch KONSISTEN dengan Keras (berdasarkan F1-score).
--- Pengujian untuk Model: cnn_max_pooling Selesai ---


--- Menguji Forward Propagation untuk Model: cnn_avg_pooling ---
1/1 ━━━━━━━━━━━━━━━━━━━━ 0

c:\Python312\Lib\site-packages\keras\src\saving\saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Bobot dari ../saved_models/cnn_avg_pooling.weights.h5 berhasil dimuat ke model Keras.
Custom CNN model built with layers from Keras model.

--- Starting Forward Propagation (from scratch) ---
--- Forward Propagation (from scratch) Finished ---

--- Perbandingan Hasil Prediksi ---
Sample true labels: [3 8 8 0 6 6 1 6 3 1]
Keras predicted labels: [5 8 0 0 4 6 1 6 3 1]
Scratch predicted labels: [5 8 0 0 4 6 1 6 3 1]

Probabilitas output Keras (sampel pertama, 5 output pertama): [0.02736873 0.00651841 0.01151324 0.2942314  0.00992508]
Probabilitas output Scratch (sampel pertama, 5 output pertama): [0.02736874 0.00651841 0.01151325 0.29423154 0.00992508]

Macro F1-Score (Keras): 0.5429
Macro F1-Score (Scratch): 0.5429
Implementasi forward propagation from scratch KONSISTEN dengan Keras (berdasarkan F1-score).
--- Pengujian untuk Model: cnn_avg_pooling Selesai ---
